## Introduction
This notebook is used to analyze the testing result and obtain the confidence score threshold.

In [ ]:
import os
import pickle
from PIL import Image
import random
import pandas as pd
from tqdm.autonotebook import tqdm
import shapely
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from datetime import datetime as dt


IoU_thresh = 0.5
r_decision_px = 12

# please change these two paths accordingly
output_figures_folder_path, detection_results_file_path = (
    # Experiment name for the figures
    'Faster-RCNN_Tr:Real-LINZ_Test:Real-LINZ',                                                     
    # Path to the elaluation results generated by MMDetection or MMYOLO
    '../work_dirs/faster-rcnn/LINZ2UGRC/faster-rcnn_train_real_linz_test_real_linz/prediction.pkl'
)

save_fig = False
if save_fig:
    os.makedirs(output_figures_folder_path, exist_ok=True)

## Load Detector Prediction file

RUN ONLY ONCE.

In [ ]:
t_start = dt.now()
print(f'Loading started: {t_start}')
with open(detection_results_file_path, 'rb') as f:
    detection_results = pickle.load(f)
t_end = dt.now()
print(f'Loading ended:   {t_end} | Duration: {t_end-t_start}')

print(f'\nNum. images: {len(detection_results):,d}')

In [ ]:
def get_bboxes_iou(bbox1, bbox2):
    if not isinstance(bbox1, shapely.Polygon):
        bbox1 = shapely.Polygon(
            [
                (bbox1[0], bbox1[1]), 
                (bbox1[2], bbox1[1]), 
                (bbox1[2], bbox1[3]), 
                (bbox1[0], bbox1[3])
            ]
        )
    
    if not isinstance(bbox2, shapely.Polygon):
        bbox2 = shapely.Polygon(
            [
                (bbox2[0], bbox2[1]), 
                (bbox2[2], bbox2[1]),
                (bbox2[2], bbox2[3]),
                (bbox2[0], bbox2[3])
            ]
        )

    intersection = bbox1.intersection(bbox2)
    union = bbox1.union(bbox2)
    
    return intersection.area / union.area

## Process Detector Predictions Data

RUN ONLY ONCE as it may take some time to process (especially if the dataset is big). After storring the GTs and PREDs data (below) you can directly load them if you want to run this code again.

In [ ]:
pred_data_all = []
gt_data_all = []
for img_res in tqdm(detection_results):
    img = Image.open(img_res['img_path'])
    
    # GTs
    gt_instances = []
    for label, bbox in zip(img_res['gt_instances']['labels'], img_res['gt_instances']['bboxes']):
        bbox_shp = shapely.Polygon([(bbox[0], bbox[1]), (bbox[2], bbox[1]), (bbox[2], bbox[3]), (bbox[0], bbox[3])])
        
        gt_instances.append(
            dict(
                label = int(label),
                bbox = np.array(bbox),
                bbox_shp = bbox_shp,
                img_path = img_res['img_path']
            )
        )

    gt_instances = pd.DataFrame(gt_instances)
    gt_data_all.append(gt_instances)
    
    # Re-format predictions
    pred_instances = []
    for label, bbox, score in zip(img_res['pred_instances']['labels'], img_res['pred_instances']['bboxes'], img_res['pred_instances']['scores']):
        # print(int(label), np.array(bbox), float(score))
        bbox_shp = shapely.Polygon([(bbox[0], bbox[1]), (bbox[2], bbox[1]), (bbox[2], bbox[3]), (bbox[0], bbox[3])])
            
        pred_instances.append(
            dict(
                label = int(label),
                bbox = np.array(bbox),
                bbox_shp = shapely.Polygon([(bbox[0], bbox[1]), (bbox[2], bbox[1]), (bbox[2], bbox[3]), (bbox[0], bbox[3])]),
                score = float(score),
                img_path = img_res['img_path'],
                is_TP = False,
                IoU = None
            )
        )

    if len(pred_instances) == 0:
        continue
        
    pred_instances = pd.DataFrame(pred_instances)
    
    # Get TPs
    IoUs_pred = []
    for idx, gt_r in gt_instances.iterrows():
        assert gt_r.label==0, 'This version of the code supports only one class: "small"'
        
        IoUs = pred_instances.bbox_shp.apply(lambda x: get_bboxes_iou(x, gt_r.bbox_shp))
        IoUs_pred.append(IoUs.to_list())
        
        mask_IoU = IoUs >= IoU_thresh
        mask_TP = pred_instances.is_TP == False
        mask = mask_IoU & mask_TP
        
        if not mask.any():
            continue

        idx = pred_instances[mask].iloc[0].name
        pred_instances.loc[idx, "is_TP"] = True
        pred_instances.loc[idx, "IoU"] = IoUs.loc[idx]

    if len(IoUs_pred) > 0:
        IoUs_pred = np.array(IoUs_pred)
        mask_IoU_missing = ~pred_instances.IoU.notna()
        pred_instances.loc[mask_IoU_missing, 'IoU'] = IoUs_pred.max(axis=0)[mask_IoU_missing]
    
    # pred_data_all.append(pred_instances.drop('bbox_shp', axis=1))
    pred_data_all.append(pred_instances)
    

gt_data_all = pd.concat(gt_data_all)

pred_data_all = pd.concat(pred_data_all).sort_values(by='score', ascending=False, ignore_index=True)
pred_data_all['is_FP']  = pred_data_all.is_TP.apply(lambda x: not x)
pred_data_all['Acc_TP'] = pred_data_all.is_TP.cumsum()
pred_data_all['Acc_FP'] = pred_data_all.is_FP.cumsum()
pred_data_all['Precision'] = pred_data_all.Acc_TP / (pred_data_all.Acc_TP + pred_data_all.Acc_FP)
pred_data_all['Recall'] = pred_data_all.Acc_TP / gt_data_all.shape[0]
pred_data_all['F1'] = 2*(pred_data_all.Precision*pred_data_all.Recall) / (pred_data_all.Precision+pred_data_all.Recall)
pred_data_all.loc[~pred_data_all.IoU.notna(),'IoU'] = 0.0

## Store/Load GTs and PREDs data

For subsequent runs of this code just load the GTs and PREDs data, saved in advance, by running the cell below and after that [Load](#Load).

In [ ]:
# Store gt_data_all and pred_data_all
output_folder_data_path = os.path.dirname(detection_results_file_path)

pred_data_all_file_path = os.path.join(output_folder_data_path, 'pred_data_all.pkl')
gt_data_all_file_path   = os.path.join(output_folder_data_path, 'gt_data_all.pkl')

In [ ]:
# STORE DATA

# Pred
t_start = dt.now()
print(f'Saving started (pred_data_all): {t_start}')
pred_data_all.to_pickle(pred_data_all_file_path)
t_end = dt.now()
print(f'Saving ended (pred_data_all):   {t_end} | {t_end-t_start}')
print()

# GT
t_start = dt.now()
print(f'Saving started (gt_data_all): {t_start}')
gt_data_all.to_pickle(gt_data_all_file_path)
t_end = dt.now()
print(f'Saving ended (gt_data_all):   {t_end} | {t_end-t_start}')

In [ ]:
# LOAD DATA

# Pred
t_start = dt.now()
print(f'Loading started (pred_data_all): {t_start}')
pred_data_all = pd.read_pickle(pred_data_all_file_path)
t_end = dt.now()
print(f'Loading ended (pred_data_all):   {t_end} | {t_end-t_start}')
print()

# GT
t_start = dt.now()
print(f'Loading started (gt_data_all): {t_start}')
gt_data_all = pd.read_pickle(gt_data_all_file_path)
t_end = dt.now()
print(f'Loading ended (gt_data_all):   {t_end} | {t_end-t_start}')

## Data visualization

In [ ]:
gt_instances

In [ ]:
pred_instances

In [ ]:
gt_data_all

In [ ]:
pred_data_all

## Calculate Evaluation Metrics
The F1 score is used as the confidence score threshold for labeling test data

In [ ]:
precision = np.array(pred_data_all.Precision.to_list() + [0])
recall    = np.array(pred_data_all.Recall.to_list() + [1])

n_point_AP_approx = 101

recall_approx = np.linspace(0, 1, n_point_AP_approx)

precision_approx = []
for recall_val in recall_approx:
    mask_recall = recall >= recall_val

    precision_approx.append(np.max(precision[mask_recall]))

precision_approx = np.array(precision_approx)

AP = np.sum(precision_approx / n_point_AP_approx)
print(f'AP: {AP:.4}  <-- This value should be the same or very similar to the evaluation from MMYOLO or MMDetection.')

i_F1_max = pred_data_all.F1.argmax()
score_F1_max, F1_max = pred_data_all[['score', 'F1']].iloc[i_F1_max].to_list()
print(f'F1_max: {F1_max:.4f} | Score thresh.: {score_F1_max:.4f}')

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(10,5))

ax = axs[0]
ax.plot([0, 1], [1, 0], lw=0.75, c='k', ls='--')
pred_data_all.plot(x='Recall', y='Precision', legend=False, label='R-P curve', ylabel='Precision', ax=ax)

ax.plot(recall_approx, precision_approx, '--r', label=f'R-P curve ({n_point_AP_approx} pts approx.)')

ax.legend()
ax.set_title(f'Precision-Recall Curve (AP: {AP:.4})', fontsize=10)
ax.axis('square')

ax = axs[1]
pred_data_all.plot(
    x='score', 
    y=['Recall', 'Precision', 'F1'],
    style=['--', '--', '-'],
    xlabel='Confidence score',
    legend=True, 
    ax=ax
)

ax.scatter([score_F1_max], [F1_max], marker='o', facecolor='none', edgecolor='red', zorder=100)

ax.annotate(
    text=f'Conf. score: {score_F1_max:.4f}\nF1: {F1_max:.4f}',
    xy=(score_F1_max, F1_max),
    xytext=(score_F1_max, F1_max*0.5),
    arrowprops=dict(
        width=0.5, 
        headwidth=5,
        edgecolor='k', 
        facecolor='k'
    ),
    c='white',
    ha='center',
    va='top',
    bbox=dict(
        edgecolor='none',
        facecolor='k',
        alpha=0.35
    )
)
ax.axis('square')
# ax.legend(loc='upper right')

ax.set_title('Precision/Recall/F1 vs. Conf. Score', fontsize=10)

fig.suptitle(f'{" | ".join(output_figures_folder_path.split("_"))}', fontweight='bold')
fig.tight_layout()

if save_fig:
    fig_file_head = os.path.join(output_figures_folder_path, 'P-R curves')
    fig.savefig(fig_file_head+'.png', dpi=200)

## [Optional] Visualize prediction statistics
Here we provide example code for visualizing positive and negative sample statistics.

In [ ]:
## Overall predictions distribution
fig, ax = plt.subplots(figsize=(6,6.7))

d = pred_data_all[pred_data_all.IoU.notna()]
d_positive = d[ d.is_TP]
d_negative = d[~d.is_TP]

scatter_plot_params = dict(
    x='score', 
    y='IoU', 
    s=6, 
    alpha=0.2,
    edgecolor='none',
    ax=ax
)
d_positive.plot.scatter(**scatter_plot_params, label=f'Positive ({d_positive.shape[0]:,d})', c='g')
d_negative.plot.scatter(**scatter_plot_params, label=f'Negative ({d_negative.shape[0]:,d})', c='magenta')

ax.axvline(score_F1_max, c='k', ls='--', lw=1)
ax.text(
    x=score_F1_max+0.01, 
    y=1-0.01, 
    s=f'Conf. score thresh. (F1 max):\n{score_F1_max:.4f}', 
    ha='left', va='top', rotation='horizontal',
)

ax.axhline(IoU_thresh, c='k', ls='--', lw=1)
ax.text(x=1-0.01, y=IoU_thresh+0.01, s=f'IoU thresh.\n{IoU_thresh:.3f}', ha='right')

ax.set_xlabel('Confidence Score')
ax.set_ylabel('Ground Truth IoU')

# ax.legend()
legend = ax.get_legend()
legend.set_title('Prediction Type')
# legend._loc_real = 'upper right'
for h in legend.legend_handles:
    h.set_alpha(1)
    h.set_sizes([10])

ax.axis('square')
ax.axis([-0.05,1.05,-0.05,1.05])

ax.set_title('All predictions distribution', fontsize=10)

new_line = '\n'
fig.suptitle(f'{new_line.join(output_figures_folder_path.split("_"))}', fontweight='bold')

fig.tight_layout()

if save_fig:
    fig_file_head = os.path.join(output_figures_folder_path, 'Predictions Distribution - All')
    fig.savefig(fig_file_head+'.png', dpi=200)

In [ ]:
## Positive predictions distribution (TP + FN)
fig, ax = plt.subplots(figsize=(6,6.7))

d_TP = pred_data_all[pred_data_all.IoU.notna() & pred_data_all.is_TP & (pred_data_all.score >= score_F1_max)]
d_FN = pred_data_all[pred_data_all.IoU.notna() & pred_data_all.is_TP & (pred_data_all.score < score_F1_max)]
print(
    f'Num. TPs: {d_TP.shape[0]:,d}', 
    f'Num. FNs: {d_FN.shape[0]:,d}', 
    f'Num. Positive Preds.: {d_TP.shape[0]+d_FN.shape[0]:,d}', 
    f'Num. GTs: {gt_data_all.shape[0]:,d}',
    sep=' | '
)

# Positives
d_TP.plot.scatter(
    x='score', 
    y='IoU', 
    c='g',
    s=6, 
    alpha=0.2,
    edgecolor='none',
    label='TP',
    ax=ax
)
ax.text(
    x=(1+score_F1_max)/2,
    y=(1+IoU_thresh)/2,
    s=f'TPs:\n{d_TP.shape[0]:,d}',
    c='white',
    ha='center', va='center',
    bbox=dict(
        edgecolor=None,
        facecolor='k',
        alpha=0.5
    )
)

d_FN.plot.scatter(
    x='score', 
    y='IoU', 
    c='lime',
    s=6, 
    alpha=0.2,
    edgecolor='none',
    label='FN',
    ax=ax
)
ax.text(
    x=(score_F1_max)/2,
    y=(1+IoU_thresh)/2,
    s=f'FNs:\n{d_FN.shape[0]:,d}',
    c='white',
    ha='center', va='center',
    bbox=dict(
        edgecolor=None,
        facecolor='k',
        alpha=0.5
    )
)


ax.axvline(score_F1_max, c='k', ls='--', lw=1)
ax.text(
    x=score_F1_max+0.01, 
    y=1-0.01, 
    s=f'Conf. score thresh. (F1 max):\n{score_F1_max:.4f}', 
    ha='left', va='top', rotation='horizontal',
)

ax.axhline(IoU_thresh, c='k', ls='--', lw=1)
ax.text(x=1-0.01, y=IoU_thresh+0.01, s=f'IoU thresh.\n{IoU_thresh:.3f}', ha='right')

ax.set_xlabel('Confidence Score')
ax.set_ylabel('Ground Truth IoU')

# ax.legend()
legend = ax.get_legend()
legend.set_title('Prediction Type')
# legend._loc_real = 'upper right'
for h in legend.legend_handles:
    h.set_alpha(1)
    h.set_sizes([10])

ax.axis('square')
ax.axis([-0.05,1.05,-0.05,1.05])

ax.set_title('Positive predictions distribution', fontsize=10)

new_line = '\n'
fig.suptitle(f'{new_line.join(output_figures_folder_path.split("_"))}', fontweight='bold')

fig.tight_layout()

if save_fig:
    fig_file_head = os.path.join(output_figures_folder_path, 'Predictions Distribution - Positive (FN+TP)')
    fig.savefig(fig_file_head+'.png', dpi=200)


In [ ]:
## Negative predictions distribution (TN + FP)
d_TN1 = pred_data_all[pred_data_all.IoU.notna()&pred_data_all.is_FP&(pred_data_all.IoU>=IoU_thresh)&(pred_data_all.score<score_F1_max)]
d_TN2 = pred_data_all[pred_data_all.IoU.notna()&pred_data_all.is_FP&(pred_data_all.IoU<IoU_thresh)&(pred_data_all.score<score_F1_max)]
d_FP1 = pred_data_all[pred_data_all.IoU.notna()&pred_data_all.is_FP&(pred_data_all.IoU>=IoU_thresh)&(pred_data_all.score>=score_F1_max)]
d_FP2 = pred_data_all[pred_data_all.IoU.notna()&pred_data_all.is_FP&(pred_data_all.IoU<IoU_thresh)&(pred_data_all.score>=score_F1_max)]
# d_FP3 = pred_data_all[~pred_data_all.IoU.notna()&pred_data_all.is_FP&(pred_data_all.score>=score_F1_max)]
# print(d_FP3.shape[0])

fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(6,6.7))

# ax = axs[0]
# Negatives
scatter_plot_params = dict(
    x='score', y='IoU', s=6, 
    alpha=0.35,
    edgecolor='none',
    ax=ax
)
d_TN1.plot.scatter(**scatter_plot_params, label='TN1', c='m', )
d_TN2.plot.scatter(**scatter_plot_params, label='TN2', c='orchid')
d_FP1.plot.scatter(**scatter_plot_params, label='FP1', c='darkmagenta')
d_FP2.plot.scatter(**scatter_plot_params, label='FP2', c='magenta')

text_params = dict(
    c='white',
    ha='center', va='center',
    bbox=dict(
        edgecolor=None,
        facecolor='k',
        alpha=0.5
    )
)
ax.text(
    x=(score_F1_max)/2,
    y=(1+IoU_thresh)/2,
    s=f'TN1:\n{d_TN1.shape[0]:,d}',
    **text_params
)
ax.text(
    x=(score_F1_max)/2,
    y=(IoU_thresh)/2,
    s=f'TN2:\n{d_TN2.shape[0]:,d}',
    **text_params
)
ax.text(
    x=(1+score_F1_max)/2,
    y=(1+IoU_thresh)/2,
    s=f'FP1:\n{d_FP1.shape[0]:,d}',
    **text_params
)
ax.text(
    x=(1+score_F1_max)/2,
    y=(IoU_thresh)/2,
    s=f'FP2: {d_FP2.shape[0]:,d}\n(@IoU0.0: {d_FP2[d_FP2.IoU==0].shape[0]:,d})',
    **text_params
)

ax.axvline(score_F1_max, c='k', ls='--', lw=1)
ax.text(
    x=score_F1_max+0.01, 
    y=1-0.01, 
    s=f'Conf. score\n(F1 max):\n{score_F1_max:.4f}', 
    ha='left', va='top', rotation='horizontal',
)

ax.axhline(IoU_thresh, c='k', ls='--', lw=1)
ax.text(x=1-0.01, y=IoU_thresh+0.01, s=f'IoU thresh.\n{IoU_thresh:.3f}', ha='right')

ax.set_xlabel('Confidence Score')
ax.set_ylabel('Ground Truth IoU')

if score_F1_max >= 0.5:
    legend_loc = 'upper left'
else:
    legend_loc = 'upper right'

legend = ax.legend(ncols=2, loc=legend_loc)
# legend = ax.get_legend()
legend.set_title('Predictions Type')
# legend.set_ncols(2)
# # legend._loc_real = 'upper right'
for h in legend.legend_handles:
    h.set_alpha(1)
    h.set_sizes([10])


ax.axis('square')
ax.axis([-0.05,1.05,-0.05,1.05])

ax.set_title('Negative predictions distribution', fontsize=10)

new_line = '\n'
fig.suptitle(f'{new_line.join(output_figures_folder_path.split("_"))}', fontweight='bold')

fig.tight_layout()

if save_fig:
    fig_file_head = os.path.join(output_figures_folder_path, 'Predictions Distribution - Negative (TN+FP)')
    fig.savefig(fig_file_head+'.png', dpi=200)

## [Optional] Visualize samples
Here we provide example code to visualize positive and negative samples

### True Positive


In [ ]:
mask_score_P = pred_data_all.score >= score_F1_max
mask_TP = pred_data_all.is_TP

mask_TP = mask_score_P & mask_TP
pred_data_TP = pred_data_all[mask_TP]

pred_data_TP.shape[0], pred_data_all.shape[0]
pred_data_TP

In [ ]:
# Randomly select one TP sample
img_path = random.choice(pred_data_TP['img_path'].values)


image = Image.open(img_path)

gt_data_img   = gt_data_all[gt_data_all.img_path==img_path]
pred_data_img = pred_data_all[pred_data_all.img_path==img_path]

fig, ax = plt.subplots(1,1,figsize=(6,6))

ax.imshow(image)

gt_color = 'cyan'
for i_gt, gt_r in gt_data_img.iterrows():
    ax.plot(*gt_r.bbox_shp.boundary.coords.xy, c=gt_color, ls='--', lw=1)
    ax.scatter(*gt_r.bbox_shp.centroid.coords.xy, fc=gt_color, marker='x', lw=1)


for i_pred, pred_r in pred_data_img.iterrows():
    if not pred_r.is_TP:
        continue

    if pred_r.score >= score_F1_max:
        pred_color = 'lime'
        
    else:
        # pred_color = 'magenta'
        pred_color = 'red'
    
    print(i_pred, pred_r.score)
    
    ax.plot(*pred_r.bbox_shp.boundary.coords.xy, c=pred_color)
    ax.scatter(*pred_r.bbox_shp.centroid.coords.xy, fc=pred_color)

    annot_params = dict(
        x=(pred_r.bbox[0]+pred_r.bbox[2])/2, 
        s=f'Conf:{pred_r.score:.2f}\nIoU:{pred_r.IoU:.2f}', 
        c=pred_color, 
        backgroundcolor=(0.,0.,0.,0.25),
        bbox=dict(edgecolor='none', facecolor='k', alpha=0.3),
        ha='center',
        fontsize=8
    )
    if pred_r.bbox_shp.centroid.coords.xy[1][0] / image.size[1] >= 0.5:
        annot_params.update(
            dict(
                y=pred_r.bbox[1]-1,
                va='bottom'
            )
        )
        
    else:
        annot_params.update(
            dict(
                y=pred_r.bbox[3]+1,
                va='top'
            )
        )
    
    ax.text(**annot_params)

# Draw edge samples grid
line_params = dict(lw=0.75, ls='dashed', c='k')
ax.axvline(r_decision_px, **line_params)
ax.axvline(image.size[0]-r_decision_px-1, **line_params)
ax.axhline(r_decision_px, **line_params)
ax.axhline(image.size[1]-r_decision_px-1, **line_params)


ax.set_title(f'True Positive:{os.path.basename(img_path)}')
ax.grid(False)

### False Negative

In [ ]:
mask_score_N = pred_data_all.score < score_F1_max
mask_TP = pred_data_all.is_TP

mask_FN = mask_score_N & mask_TP
pred_data_FN = pred_data_all[mask_FN]
pred_data_FN

In [ ]:
# Randomly select one FN sample
img_path = random.choice(pred_data_FN['img_path'].values)
image = Image.open(img_path)


gt_data_img   = gt_data_all[gt_data_all.img_path==img_path]
pred_data_img = pred_data_all[pred_data_all.img_path==img_path]

fig, ax = plt.subplots(1,1,figsize=(6,6))

ax.imshow(image)

for i_gt, gt_r in gt_data_img.iterrows():
    ax.plot(*gt_r.bbox_shp.boundary.coords.xy, c='lime', ls='--', lw=1)
    ax.scatter(*gt_r.bbox_shp.centroid.coords.xy, fc='lime', marker='x', lw=1)


for i_pred, pred_r in pred_data_img.iterrows():
    if not pred_r.is_TP:
        continue

    if pred_r.score >= score_F1_max:
        pred_color = 'lime'
    else:
        pred_color = 'red'
    
    print(i_pred, pred_r.score)
    
    ax.plot(*pred_r.bbox_shp.boundary.coords.xy, c=pred_color)
    ax.scatter(*pred_r.bbox_shp.centroid.coords.xy, fc=pred_color)
    # ax.text(pred_r.bbox[0], pred_r.bbox[1]-2, f'Conf:{pred_r.score:.4f}', c='red')
    annot_params = dict(
        x=(pred_r.bbox[0]+pred_r.bbox[2])/2,
        s=f'Conf:{pred_r.score:.3f}\nIoU:{pred_r.IoU:.3f}',
        c=pred_color, 
        # backgroundcolor=(0.,0.,0.,0.25),
        bbox=dict(edgecolor='none', facecolor='k', alpha=0.35),
        ha='center',
        fontsize=8
    )
    if pred_r.bbox_shp.centroid.coords.xy[1][0] / image.size[1] >= 0.5:
        annot_params.update(
            dict(
                y=pred_r.bbox[1]-1,
                va='bottom'
            )
        )
        
    else:
        annot_params.update(
            dict(
                y=pred_r.bbox[3]+1,
                va='top'
            )
        )
    
    ax.text(**annot_params)

# Draw edge samples grid
line_params = dict(lw=0.75, ls='dashed', c='k')
ax.axvline(r_decision_px, **line_params)
ax.axvline(image.size[0]-r_decision_px-1, **line_params)
ax.axhline(r_decision_px, **line_params)
ax.axhline(image.size[1]-r_decision_px-1, **line_params)

ax.set_title(f'False Negative:{os.path.basename(img_path)}')
ax.grid(False)

### False Positive

In [ ]:
mask_score_P = pred_data_all.score >= score_F1_max
mask_IoU1    = pred_data_all.IoU.notna()
mask_FP      = pred_data_all.is_FP

mask_FP1 = mask_score_P & mask_FP & mask_IoU1
pred_data_FP1 = pred_data_all[mask_FP1]
pred_data_FP1

In [ ]:
# Randomly select one FP sample
img_path = random.choice(pred_data_FP1['img_path'].values)

image = Image.open(img_path)

gt_data_img   = gt_data_all[gt_data_all.img_path==img_path]
pred_data_img = pred_data_all[pred_data_all.img_path==img_path]

fig, ax = plt.subplots(1,1,figsize=(6,6))

ax.imshow(image)

for i_gt, gt_r in gt_data_img.iterrows():
    ax.plot(*gt_r.bbox_shp.boundary.coords.xy, c='lime', ls='--', lw=1)
    ax.scatter(*gt_r.bbox_shp.centroid.coords.xy, fc='lime', marker='x', lw=1)


for i_pred, pred_r in pred_data_img.iterrows():
    # if not pred_r.is_TP:
    #     continue
    
    if pred_r.score < score_F1_max:
        continue

    if pred_r.is_TP:
        # continue
        pred_color = 'lime'
        label = 'TP'
        
    else:
        pred_color = 'red'
        label = 'FP'
    
    print(i_pred, pred_r.score)
    
    ax.plot(*pred_r.bbox_shp.boundary.coords.xy, c=pred_color)
    ax.scatter(*pred_r.bbox_shp.centroid.coords.xy, fc=pred_color)
    
    # ax.text(pred_r.bbox[0], pred_r.bbox[1]-2, f'Conf:{pred_r.score:.4f}', c='red')
    annot_params = dict(
        # x=pred_r.bbox[0], 
        x=pred_r.bbox_shp.centroid.x,
        s=f'[{label}] Conf:{pred_r.score:.4f}',
        c=pred_color,
        ha='center',
        backgroundcolor=(0.,0.,0.,0.25),
        fontsize=8
    )
    if pred_r.bbox_shp.centroid.coords.xy[1][0] / image.size[1] >= 0.5:
        annot_params.update(
            dict(
                y=pred_r.bbox[1]-1,
                va='bottom'
            )
        )
        
    else:
        annot_params.update(
            dict(
                y=pred_r.bbox[3]+1,
                va='top'
            )
        )
    
    ax.text(**annot_params)


# Draw edge samples grid
line_params = dict(lw=0.75, ls='dashed', c='k')
ax.axvline(r_decision_px, **line_params)
ax.axvline(image.size[0]-r_decision_px-1, **line_params)
ax.axhline(r_decision_px, **line_params)
ax.axhline(image.size[1]-r_decision_px-1, **line_params)


ax.set_title(f'False Positive:{os.path.basename(img_path)}')
ax.grid(False)